<a href="https://colab.research.google.com/github/vapemx/AeroFeel/blob/main/AeroFeel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AeroFeel
###### by: Datahacks
-----------------------

El mercado aéreo comercial de México es altamente competitivo, lo que hace que la retención de clientes sea un aspecto crucial para alcanzar el éxito en este sector.

## Reto

VivaAerobus nos entregó un dataset en formato xlsx con 4 hojas, las cuáles están separadas por el punto en el que se recabó la opinión del cliente (paypoint). Este dataset con aproximadamente 76 mil registros en total, fue utilizado para el entrenamiento y evaluación de un modelo NLP, sin embargo, este notebook está creado con el objetivo de introducir la información que se recaba día con día. 

## Solución

Para la solución del reto decidimos utiizar DistilBERT, que es una versión más ligera y rápida del modelo original BERT, el cuuál es un modelo de procesamiento del lenguaje natural. 

Utilizamos este modelo para un análisis de sentimiento, pero, para realizar el fine-tuning, comparamos la calificación dada por el cliente con el sentimiento dado por el modelo, para después compararlo en una matriz de confusión.

Posteriormente, los comentarios se clasifican en las diferentes áreas de interés para la aerolínea con base en palabras clave; finalmente, se hace una limpieza de palabras altisonantes.

## Resultados

Con un promedio de 83% de precisión en los conjuntos de validación, tenemos un análisis de sentimiento satisfactorio gracias a nuestro modelo.

Para este punto, en el dataset final ya tenemos el paypoint, sentimiento y clasificacón, así como todos los otros campos previamente dados. 

Con esta información, se le permite a la aerolínea personalizar la búsqueda de comentarios ya sea por área, sentimiento, clasificación y hasta rango de fechas, con el objetivo de enfocarse específicamente en las áreas de oportunidad.

# Imports e installs

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
from datasets import Dataset
import matplotlib.pyplot as plt
from transformers import pipeline
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification

## Habilitación de aceleración por GPU

**Nota: habilitar la aceleración por GPU de google colab**

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

# Funciones de limpieza
1. Identificar columnas vacias y borralas.
2. Contar registros vacios.
3. Summary de cuantos datos hay en cada columna.
4. Renombrar columnas.

In [ ]:
def changeDateWithoutHours(df, nameColumn='Date'):
    df[nameColumn] = pd.to_datetime(df[nameColumn])
    df[nameColumn] = df[nameColumn].dt.date
    return df


In [ ]:
print(df.info())

In [ ]:
def searchNullColumns(df):
  nullColumns = df.columns[df.isnull().any()].tolist()
  if len(nullColumns) > 0:
      print("Se encontraron columnas con filas vacias")
      return nullColumns
  else:
      print("No se encontraron columnas con valores vacios en el dataset.")
      return None


In [ ]:
def cleanColumns(df,columns):
  df = df.drop(columns, axis=1)


In [ ]:
df.info()

# Import xlsx dataset

In [ ]:
input = input("Ingrese la ruta del archivo xlsx: ")

df_booking = pd.read_excel(input, sheet_name="Booking flow")
df_checkin = pd.read_excel(input, sheet_name="Checkin")
df_rsv = pd.read_excel(input, sheet_name="Manage my booking")
df_feedback = pd.read_excel(input, sheet_name="Feedback button")

# Clean and analyze data

In [ ]:
df_checkin['Comment'].fillna("N/A", inplace=True)

In [ ]:
df_booking = pd.read_csv(booking)
df_booking.head()
df_booking = changeDateWithoutHours(df_booking)

In [ ]:
nullCol_booking= searchNullColumns(df_booking)
fNull_booking = df_booking["text"].isnull().sum()
fNull_booking

In [ ]:
df_booking["text"] = df_booking["text"].fillna("NA")

In [ ]:
df_booking

In [ ]:
df_rsv = pd.read_csv(rsv)
df_rsv.head()

In [ ]:
df_rsv = changeDateWithoutHours(df_rsv)
nullCol_rsv= searchNullColumns(df_rsv)
fNull_rsv = df_rsv["_C_mo_podemos_mejorar_"].isnull().sum()
fNull_rsv

In [ ]:
df_rsv["_C_mo_podemos_mejorar_"] = df_rsv["_C_mo_podemos_mejorar_"].fillna("NA")

In [ ]:
df_booking["nps_scaled"] = df_booking["nps"]//2

In [ ]:
df_rsv["nps_scaled"] = df_rsv["nps"]//2

## Cleaning feedback dataframe

La tabla "Feedback button" cuenta con una estructura completamente diferente a las otras, por lo que se necesita limpiar y ordenar de forma especial.

In [ ]:
df_feedback.head()

In [ ]:
plt.bar(df_feedback.columns, df_feedback.count())
plt.ylabel('Cantidad de Registros')
plt.xticks(rotation = 90)
plt.show()

In [ ]:
df_feedback = df_feedback.dropna(axis=1, how='all')
# Eliminamos columnas con menos de 10 registros
df_feedback = df_feedback.loc[:, df_feedback.notnull().sum() >= 1000]

#Eliminamos una columna de relleno
df_feedback.drop('Image', axis=1, inplace=True)

#Renombramos y modificamos valores de las columnas para evitar registros nulos
df_feedback = df_feedback.rename(columns={'Unnamed: 2': 'Comments', 'Comment': 'Date'})
df_feedback['Page Load Time'] = df_feedback['Page Load Time'].str.replace('s', '')
df_feedback['Page Load Time'] = df_feedback['Page Load Time'].fillna(value=3000)
df_feedback['Page Load Time'] = df_feedback['Page Load Time'].astype(int)
df_feedback['Page Load Time'] = df_feedback['Page Load Time'].replace(3000, np.nan)
df_feedback['Comments'].fillna('Ninguno', inplace=True)
df_feedback['Page Load Time'].fillna(df_feedback['Page Load Time'].mean(), inplace=True)
df_feedback['feedback_type'].fillna('Unknown', inplace=True)
df_feedback['basket_id'] = df_feedback['basket_id'].fillna(value=False).astype(bool)
df_feedback['¿Qué te gustaría compartir con nosotros?'].fillna('Nada', inplace=True)
df_feedback['motivo_visita'].fillna('Unknown', inplace=True)
df_feedback['share_data'].fillna('no', inplace=True)
df_feedback['share_data'].replace(['si', 'yes'], True, inplace=True)
df_feedback['share_data'].replace('no', False, inplace=True)
df_feedback['error_type'].fillna('Unknown', inplace=True)
df_feedback.head(5)


df_feedback['Rating'].value_counts()
df_feedback = df_feedback.rename(columns={'Rating':'nps_scaled'})
df_feedback.isnull().sum()

# Análisis de sentimiento

In [ ]:
def analyze(df):
  # Dividir el conjunto de datos en entrenamiento y validación
  train, val = train_test_split(df, test_size=0.2, random_state=42)

  # Inicializa el tokenizador
  tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-multilingual-cased")

  # Tokenizar el texto
  train_encodings = tokenizer(list(train['Comments']), truncation=True, padding=True)
  val_encodings = tokenizer(list(val['Comments']), truncation=True, padding=True)

  # Cargar el pipeline de análisis de sentimiento
  sentiment_analysis = pipeline(
      "sentiment-analysis",
      model="nlptown/bert-base-multilingual-uncased-sentiment",
      tokenizer="nlptown/bert-base-multilingual-uncased-sentiment",
  )

  # Función para predecir el sentimiento de un comentario
  def predict_sentiment(comment):
      # Truncar el comentario si excede 512 tokens
      truncated_comment = comment[:512]
      
      result = sentiment_analysis(truncated_comment)
      sentiment = result[0]['label'].split('_')[-1].lower()
      return sentiment

  # Clasifica los comentarios en la columna 'Comments'
  df['Sentiment'] = df['Comments'].apply(predict_sentiment)

  class FeedbackDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

  # Crear los conjuntos de datos de entrenamiento y validación
  train_labels = train['Comments'].map({'positive': 0, 'negative': 1, 'neutral': 2}).tolist()
  val_labels = val['Comments'].map({'positive': 0, 'negative': 1, 'neutral': 2}).tolist()
  train_dataset = FeedbackDataset(train_encodings, train_labels)
  val_dataset = FeedbackDataset(val_encodings, val_labels)

  import pandas as pd
  from datasets import Dataset
  from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
  from transformers import Trainer, TrainingArguments
  from sklearn.metrics import accuracy_score

  # 1. Cargar y preparar los datos
  # Suponiendo que ya tienes un dataframe 'df' con las columnas 'Comments' y 'Sentiment'

  # Renombrar las columnas
  df = df.rename(columns={'Comments': 'text', 'Sentiment': 'label'})

  # Crear un diccionario para convertir las etiquetas a números
  label_to_id = {
      '1 star': 0, 
      '2 stars': 1, 
      '3 stars': 2,
      '4 stars': 3,
      '5 stars': 4
  }
  df['label'] = df['label'].apply(lambda x: label_to_id[x])

  # Dividir los datos en conjuntos de entrenamiento y validación (ajusta la proporción según lo necesario)
  train_df = df.sample(frac=0.8, random_state=42)
  val_df = df.drop(train_df.index)

  # Convertir los dataframes de pandas a datasets de Hugging Face
  train_dataset = Dataset.from_pandas(train_df)
  val_dataset = Dataset.from_pandas(val_df)

  # 2. Preparar el tokenizador y el modelo
  tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased')
  model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=5)

  # 3. Tokenizar y codificar los conjuntos de datos
  max_length = 128

  def encode_examples(examples):
      encoded = tokenizer(
          examples['text'], 
          truncation=True, 
          padding='max_length', 
          max_length=max_length
      )
      
      labels = examples['label']
      encoded.update({'labels': labels})
      
      return encoded

  train_dataset = train_dataset.map(encode_examples, batched=True)
  val_dataset = val_dataset.map(encode_examples, batched=True)

  # 4. Configurar y entrenar el Trainer
  def compute_metrics(pred):
      labels = pred.label_ids
      preds = pred.predictions.argmax(-1)
      acc = accuracy_score(labels, preds)
      return {'accuracy': acc}

  training_args = TrainingArguments(
      output_dir='./results',
      num_train_epochs=3,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      logging_dir='./logs',
      logging_steps=10,
      evaluation_strategy="steps",
      save_strategy="steps",
      save_steps=50,
      load_best_model_at_end=True,
      metric_for_best_model="accuracy",
      seed=42,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=val_dataset,
      compute_metrics=compute_metrics,
  )


  trainer.train()


  # Guardar el modelo entrenado y el tokenizador
  trainer.save_model("sentiment_analysis_multilingual")
  tokenizer.save_pretrained("sentiment_analysis_multilingual")

  # Evaluar el modelo en el conjunto de datos de entrenamiento
  train_eval_results = trainer.evaluate(train_dataset)

  print("Resultados de la evaluación en el conjunto de entrenamiento de:")
  print(train_eval_results)

  # Evaluar el modelo en el conjunto de datos de validación
  eval_results = trainer.evaluate()

  print("Resultados de la evaluación en el conjunto de validación:")
  print(eval_results)

  def custom_sentiment_pipeline(text, model, tokenizer, negative_words):
    # Verificar si alguna palabra negativa está presente en el texto
    if any(word.lower() in text.lower() for word in negative_words):
        return "1 star"

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=-1).item()

    return id_to_label[predictions]

  # Cargar el modelo afinado y el tokenizador
  model = AutoModelForSequenceClassification.from_pretrained("sentiment_analysis_multilingual")
  tokenizer = DistilBertTokenizerFast.from_pretrained("sentiment_analysis_multilingual")

  # Diccionario de conversiones de ID a etiqueta
  id_to_label = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5}

  # Lista de palabras negativas
  negative_words = ["malo", "terrible", "horrible", "pésimo", "asqueroso", "chinguen", "pendejos"]

  # Comentario de ejemplo en español
  comentario = "horrible"

  # Probar el pipeline con el comentario de ejemplo
  sentiment_label = custom_sentiment_pipeline(comentario, model, tokenizer, negative_words)

  print(f"Comentario: {comentario}")
  print(f"Sentimiento: {sentiment_label}")

  return df

## Get Predicted Data

In [ ]:
chekin = analyze(df_checkin)
feedback = analyze(df_feedback)
rsv = analyze(df_rsv)
booking = analyze(df_booking)

# Categorize data

In [ ]:
checkin['paypoint'] = 'checkin'
feedback['paypoint'] = 'feedback'
rsv['paypoint'] = 'rsv'
booking['paypoint'] = 'booking'

In [ ]:
dataset = pd.concat([checkin, feedback, rsv, booking], ignore_index=True)
dataset = dataset.fillna('None')

In [ ]:
# Diferentes categorías y palabras clave que nos pueden interesar para clasificar los comentarios

categories = {
    'Pagos': ['confirmacion', 'cancelacion', 'pago', 'pagos', 'tarjeta', 'banco', 'credito', 'debito', 'TDC', 'tdc', 'rechazo', 'declinado',
              'Boleto', 'Pasaje', 'Asiento', 'Ventanilla', 'Pasillo', 'Facturación', 'Equipaje', 'Sobrepeso', 'Maleta', 'Mochila', 
              'Bolsa de viaje', 'Pago', 'Efectivo', 'Tarjeta de crédito', 'Tarjeta de débito', 'Transferencia bancaria', 'PayPal', 'Google Pay',
              'Apple Pay', 'Moneda local', 'Cambio de moneda', 'Tasa de cambio', 'Impuestos', 'Seguro de viaje', 'Seguro de cancelación',
              'Seguro de equipaje', 'Seguro médico', 'Descuento', 'Oferta', 'Promoción', 'Cupón', 'Código de descuento', 'Equipaje perdido', 'Carga',
              'Tarifa de carga', 'Espacio de carga'],

    'Precios': ['caro', 'costoso', 'asientos', 'tarifas', 'tarifa', 'precio', 'precios', 'equipaje', 'peso', 'sobrepeso', 'Asiento', 'Ventanilla',
                'Pasillo', 'Equipaje', 'Sobrepeso', 'Maleta', 'Mochila', 'Bolsa de viaje', 'Tarifa de uso del aeropuerto', 'tua', 'barato', 'extras'],

    'Técnicos': ['errores', 'carga', 'traba', 'trabado', 'trabada', 'idioma'], # Web
    
    'Operaciones': ['conexion', 'escala', 'escalas', 'vuelo', 'vuelos', 'retraso', 'retrasos', 'cancelacion', 'cancelaciones',
                    'cancelado', 'cancelada', 'cancelados', 'canceladas', 'demora', 'demoras', 'demorado', 'demorada', 'demorados',
                    'demoradas', 'demorar', 'demore', 'demoren', 'accesibilidad'],

    'Reservacion': ['cambios', 'reserva', 'asiento', 'asientos', 'reasignacion', 'cambio', 'Clase', 'Asiento', 'Ventanilla', 'Pasillo',  'Equipaje',
                    'Sobrepeso', 'Maleta', 'Mochila', 'Bolsa de viaje', 'Confirmación de reserva', 'Cancelación de reserva', 'Cambio de reserva', 'Reembolso',
                    'Política de cancelación', 'Política de cambio', 'Política de reembolso', 'Política de equipaje', 'Tamaño del equipaje', 'Peso del equipaje',
                    'Equipaje de mano', 'Equipaje documentado', 'mascota'],

    'Personal': ['Desinterés', 'Impaciencia', 'Arrogancia', 'Desgano', 'Indiferencia', 'Hostilidad', 'Desorganización', 'Indolencia', 'Antipatía',
                 'Insolencia', 'Descortesía', 'Falta de empatía', 'Impuntualidad', 'Intransigencia', 'Falta de compromiso', 'Despreocupación', 'Indecisión',
                 'Falta de cooperación', 'Desprestigio', 'Falta de comunicación', 'Inflexibilidad', 'Desconfianza', 'Deshonestidad', 'Falta de iniciativa',
                 'Insensibilidad', 'Prepotencia', 'Desconcentración', 'Inatención', 'Desmotivación', 'Falta de respeto', 'Desconocimiento del trabajo',
                 'Falta de ética', 'Falta de entusiasmo', 'Falta de paciencia', 'Falta de sinceridad', 'Inflexibilidad en políticas',
                 'Incapacidad para resolver problemas', 'Falta de soluciones', 'Mala actitud hacia el trabajo', 'Falta de atención en detalles',
                 'Desorden en tareas', 'Falta de habilidad para trabajar bajo presión', 'Falta de capacidad de trabajo en equipo', 'Desinteresado',
                 'Impuntual', 'Descortés', 'Incompetente', 'Desorganizado', 'Desatento', 'Arrogante', 'Descuidado', 'Maleducado', 'Insensible', 'Irresponsable',
                 'Inflexible', 'Indiferente', 'Indolente', 'Despreocupado', 'Desmotivado', 'Despreparado', 'Malhumorado', 'Desagradable', 'Indeciso',
                 'Desconsiderado', 'Desleal', 'Insuficiente', 'Incumplido', 'Irrespetuoso', 'Desesperado', 'Deshonesto', 'Desaliñado', 'Desmotivante',
                 'Desmotivador', 'Indiscreto', 'Indignado', 'Desenfocado', 'Desatendido', 'Desentendido', 'Despistado', 'Desaprensivo', 'Desobligante',
                 'Descarado', 'Deshonrado', 'Desobediente', 'Atento', 'Cortés', 'Amable', 'Empático', 'Servicial', 'Proactivo', 'Colaborativo', 'Responsable',
                 'Detallista', 'Preciso', 'Consciente', 'Creativo', 'Disponible', 'Espontáneo', 'Generoso', 'Innovador', 'Interesado', 'Inspirador', 'Leal',
                 'Seguro', 'Tranquilo', 'Orientado al cliente', 'Positivo', 'Optimista', 'Emprendedor', 'Reflexivo', 'Solidario', 'Comunicativo', 'Trabajador',
                 'Confiado', 'Visionario', 'Dinámico', 'Persistente', 'Flexible', 'Convincente', 'Confiable', 'Alegre', 'Comprensivo', 'Paciente', 'Eficiente',
                 'Profesional', 'Sincero', 'Agradable', 'Respetuoso', 'Entusiasta', 'Organizado', 'Innovador', 'Interesado', 'Inspirador', 'Solidario'],

    'Aeropuerto': ['aeropuerto', 'seguridad', 'Transporte de equipaje', 'perdida', 'Equipaje retrasado', 'control', 'aduana', 'inmigracion', 'migracion', 'sala de espera', 'sala de abordar', 'capilla'],

    'Servicios': ['comida', 'a bordo', 'servicio',  'Wi-Fi', 'wifi', 'internet', 'bebidas', 'refrescos', 'jugos', 'fria', 'caliente', 'sabor', 'rica', 'mala', 'baños', 'luces', 'enchufes', 'enchufe', 'pantalla', 'entretenimiento'],

    'Experiencias': ['tiempo'],

    'Check-in': ['pase', 'pase de abordar', 'abordar', 'abordaje', 'check-in', 'checkin'],

    'Otros': ['identificacion', 'accesibilidad', 'discapacidad', 'movilidad', 'discapacitado', 'dicapacitada', 'discapacitados', 'documentacion', 'extras']
}

In [ ]:
dataset['category'] = 'General'
for idx, row in dataset.iterrows():
    text = row['text']

    for key, val_list in categories.items():
        for val in val_list:
            if val.upper() in text.upper():
                dataset.at[idx, 'category'] = key
                break

# Filtro de palabras altisonantes

In [ ]:
import requests

# Diccionario de palabras altisonantes
url = 'https://raw.githubusercontent.com/EddieSharp/Insultos/master/diccionario.txt'

response = requests.get(url)
content = response.text

altisonantes = content.split('\n')

In [ ]:
for idx, row in dataset.iterrows():
    text = row['text']

    for palabra in altisonantes:
        if palabra.upper() in text.upper():
            row['text'].replace(palabra, '***')